## Import Libraries

In [12]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [13]:
import matplotlib.pyplot as plt
plt.ion()

In [14]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [15]:
from os import path, pardir

## Compile and program project

In [16]:
project_path = [path.curdir, "FFT-S"]
project_path

['.', 'FFT-S']

In [17]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:8415
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x00005808.
[DEBUG] Memory segment base written at 0x00005808. Size = 0x00000068.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

In [18]:
live_plot = True

Create a figure for the plot.

In [30]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [31]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_fft"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment FFT",
}

Stop criteria to pass to the logger:

In [32]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [33]:
def stop_function(pin_values):
    return all(pin_values)

In [53]:
repetitions = 100

Perform the measurement.

In [54]:
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
        
charges_allocate = []
charges_fft = []
charges_free = []
times_allocate = []
times_fft = []
times_free = []


with DGILibExtra(**config_dict) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(10,stop_fn)
        charge2, time2 = power_and_time_per_pulse(dgilib.data, 2, stop_function=stop_function)
        charge3, time3 = power_and_time_per_pulse(dgilib.data, 3, stop_function=stop_function)
        
        if len(charge2) != 2 or len(time2) != 2 or len(charge3) != 1 or len(time3) != 1:
            print(f"Parsing of measurement data failed, charge2:{charge2}, time2:{time2}, charge3:{charge3}, time3:{time3},")
            break
        charges_allocate.append(charge2[0])
        charges_fft.append(charge3[0])
        charges_free.append(charge2[1])
        times_allocate.append(time2[0])
        times_fft.append(time3[0])
        times_free.append(time2[1])
        # dgilib.logger.plotobj.ax.set_title(f"Previous sample: charge: {sum(charge2+charge3)*1e3:.06} mC, time: {sum(time2+time3):.06} s")
        dgilib.empty_data()

    charges = charges_allocate + charges_fft + charges_free 
    times = times_allocate + times_fft + times_free

#     dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

In [60]:
import json

config = {}
config["name"] = "FFT 128"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Allocate", "Free"], 3: ["FFT"]}, 
                      "result_types": ["Charge", "Time"], 
                      "section_types": {"init": ["Allocate FFT"], 
                                        "loop": ["FFT"],
                                        "exit": ["Free FFT"]}}

with open("repeated_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

In [56]:
import pandas as pd

In [57]:
df = pd.DataFrame({'charges_allocate': charges_allocate, 'charges_fft':charges_fft, 'charges_free':charges_free, 'times_allocate':times_allocate, 'times_fft':times_fft, 'times_free':times_free})

In [58]:
df

,charges_allocate,charges_fft,charges_free,times_allocate,times_fft,times_free
0,0.000247,0.000033,0.000000e+00,0.504160,0.075308,0.000000
1,0.000247,0.000056,2.354028e-08,0.504229,0.125926,0.000066
2,0.000247,0.000056,2.110491e-08,0.504243,0.125922,0.000066
3,0.000247,0.000056,2.110644e-08,0.504233,0.125932,0.000066
4,0.000247,0.000056,2.191522e-08,0.504247,0.125929,0.000066
5,0.000247,0.000056,0.000000e+00,0.504257,0.125866,0.000000
6,0.000247,0.000033,0.000000e+00,0.504256,0.075570,0.000000
7,0.000247,0.000056,2.435083e-08,0.504326,0.125934,0.000066
8,0.000247,0.000056,2.191557e-08,0.504320,0.125930,0.000066
9,0.000247,0.000056,2.597338e-08,0.504257,0.125932,0.000066


In [59]:
df.describe()

,charges_allocate,charges_fft,charges_free,times_allocate,times_fft,times_free
count,1.000000e+02,100.000000,1.000000e+02,100.000000,100.000000,100.000000
mean,2.473384e-04,0.000052,1.648559e-08,0.504275,0.118381,0.000048
std,1.122274e-07,0.000008,1.013856e-08,0.000054,0.018016,0.000029
min,2.470802e-04,0.000033,0.000000e+00,0.504130,0.075040,0.000000
25%,2.472613e-04,0.000056,0.000000e+00,0.504243,0.125863,0.000000
50%,2.473473e-04,0.000056,2.191522e-08,0.504258,0.125926,0.000066
75%,2.474038e-04,0.000056,2.272768e-08,0.504317,0.125931,0.000066
max,2.475980e-04,0.000056,2.597464e-08,0.504386,0.125991,0.000066
